In [27]:
from constants import *
import subprocess
import json
import os
from web3 import Web3
from bit import *
from dotenv import load_dotenv

load_dotenv("example.env")

from web3.middleware import geth_poa_middleware
from eth_account import Account

from pathlib import Path
from getpass import getpass

from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI

In [28]:
mnemonic = os.getenv('MNEMONIC')

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
w3.eth.getBalance("0x0172DBb6C9583D8D4457547016Ad8BfaCDDd138D")
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
from web3.gas_strategies.time_based import medium_gas_price_strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [29]:
def derive_wallets(mnemonic, coin, numderive):
    command = f"C:/xampp/php/php.exe ./hd-wallet-derive/hd-wallet-derive.php -g --coin={coin} --numderive={numderive} --cols=path,address,privkey,pubkey --format=json --mnemonic='{mnemonic}'"
    print(command)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys=json.loads(output)
    return keys

derive_wallets("note absent option glide fossil volcano hire cabin cup diamond pond project", "BTC", 3)

C:/xampp/php/php.exe ./hd-wallet-derive/hd-wallet-derive.php -g --coin=BTC --numderive=3 --cols=path,address,privkey,pubkey --format=json --mnemonic='note absent option glide fossil volcano hire cabin cup diamond pond project'


[{'path': "m/44'/0'/0'/0/0",
  'address': '1NWzLmD9UsHV65rghzaKcG5BJCUER9LftD',
  'privkey': 'KzSQafP5XfCk2h3S1fzRQjbwTJTd8yhpTw5eZm9MHT1J7AHrZjdf',
  'pubkey': '031badabef86cfb6ae50f2cbbb62938273572dfeaf8569cc5ed71963250c462889'},
 {'path': "m/44'/0'/0'/0/1",
  'address': '17ee93yMQhuAdL38y4XDhX2ipfH3hJ64o5',
  'privkey': 'L2MiFJXRuwpmACRUNCYTJg8dgGyhNRZGSHv8zXi8kHYyMufTBU8m',
  'pubkey': '02d349cf746f7247ada535403258b77d2d6fdead36ae7e2e7c4639d4ebc714f7ff'},
 {'path': "m/44'/0'/0'/0/2",
  'address': '1KHkcfTJpmpEwP9tPSPWGnyBsQxLQ8sgRZ',
  'privkey': 'L2irnLKaYAZxVXSQkYTjmbCwmCoYa5Pfhef3DconAFv4kGgVPfhY',
  'pubkey': '03a7e63b1a46988d52e03c3ccfe6c78518b2262d3aed4f7599f4480b4fb03fea78'}]

In [30]:
coins = {ETH:derive_wallets(mnemonic,'eth', 3), BTCTEST:derive_wallets(mnemonic,'btc-test', 3)}

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

C:/xampp/php/php.exe ./hd-wallet-derive/hd-wallet-derive.php -g --coin=eth --numderive=3 --cols=path,address,privkey,pubkey --format=json --mnemonic='boring boil arrive impact chunk loud venture edge sadness kiss message powder'
C:/xampp/php/php.exe ./hd-wallet-derive/hd-wallet-derive.php -g --coin=btc-test --numderive=3 --cols=path,address,privkey,pubkey --format=json --mnemonic='boring boil arrive impact chunk loud venture edge sadness kiss message powder'


In [31]:
ETH_key = priv_key_to_account(ETH, '9cb92f6a3745a47f20f9c4f0fa93d3457959bf1c6de442e78ee3b33833540e96')

In [32]:
# CREATE TRANSACTION
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return trx_data
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [33]:
# SEND TRANSACTION
def send_tx(coin, account, recipient, amount):
    tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [34]:
send_tx(ETH, ETH_key, "0xaEF84104518e1C2a4Af7Eea85429bBd05c93a9C9", 1)

'0x5287c165f219c08480336b368fb6200233ccdc83251c002901a44e10d842032f'

In [35]:
w3.eth.get_balance(ETH_key.address)

99999579999999999999

In [36]:
print(ETH_key.address)

0x20569f4F5723BB2F7b08f241fDa1030C42d3C7EA
